In [1]:
#------------------------------------------------------------------------------------------------------------
# cosine 유사도를 이용한 BERT 추출 요약 예제
# 출처 : https://blog.devgenius.io/bert-for-text-summarization-in-python-4c527179cd98
# - BERT 문장임베딩을 이용하여, 각 문장과 다른 문장간 유사도를 계산하고, 가장 높은 유사도 점수를 가지는 문장을 선택함.
#------------------------------------------------------------------------------------------------------------
import time
from os import sys
sys.path.append('..')
from myutils import GPU_info, seed_everything, mlogging, bi_encoder
from transformers import BertTokenizer, BertModel

device = GPU_info()
seed_everything(111)
logger = mlogging(loggername="summarizer", logfilename="../../log/summarizer")


out_dimension = 0  # # 차원 축소 => 768은 0으로 입력, 128=128 입력
bi_encoder_path = "../../data11/model/kpf-sbert-v1.1" #"bongsoo/kpf-sbert-v1.1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
pooling_mode = 'mean' # bert면=mean, albert면 = cls

word_embedding_model1, bi_encoder1 = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                                pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
  
print(f'\n---bi_encoder---------------------------')
print(bi_encoder1)
print(word_embedding_model1)
#------------------------------------------------------------------------------------------------------------------------


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
logfilepath:../../log/summarizer_2023-06-20.log

---bi_encoder---------------------------
SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)
Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 


In [2]:
from myutils import embed_text

paragraphs = [
    "Text 요약은 중요한 정보를 유지하면서 텍스트 문서의 길이를 줄이는 프로세스입니다.",
    "이는 추출 기반 요약 및 추상화 기반 요약을 포함한 다양한 방법을 통해 달성할 수 있습니다. ",
    "추출 요약을 위한 인기 있는 접근 방식 중 하나는 BERT로 , 이는 텍스트 요약 작업을 위해 미세 조정할 수 있는 사전 훈련된 언어 모델입니다.",
    "이 자습서에서는 Python에서 텍스트 요약에 BERT를 사용하는 방법을 살펴봅니다.",
    "구현을 시작하기 전에 자연어 처리(NLP) , Python 프로그래밍 언어 및 PyTorch 라이브러리에 대한 기본적인 이해가 필수적입니다.",
    "또한 BERT 및 Hugging Face Transformers 라이브러리 에 대한 경험이 있으면 도움이 될 것입니다 .",
    "터미널에서 다음 명령을 실행하여 필요한 라이브러리를 설치할 수 있습니다.",
    "이제 Python에서 텍스트 요약을 위해 BERT를 구현할 준비가 되었습니다.",
    "다음으로 Hugging Face 변환기 라이브러리를 사용하여 사전 훈련된 BERT 모델과 토크나이저를 로드합니다.",
    "BERT 모델에 텍스트를 입력하기 전에 사전 처리가 필요합니다. ",
    "먼저 NLTK sent_tokenize 함수를 사용하여 텍스트를 문장으로 분할합니다.",
    "다음으로 BERT 토크나이저를 사용하여 각 문장을 토큰화합니다.",
    "토큰화된 문장을 BERT 모델에 공급하려면 BERT 토크나이저를 사용하여 인코딩해야 합니다. ",
    "또한 각 입력에 [CLS](문장 시작) 및 [SEP](문장 끝)와 같은 특수 토큰을 추가해야 합니다.",
    "입력을 인코딩하면 BERT 모델에 입력하여 문장 임베딩을 생성할 수 있습니다.",
    "BERT 모델의 마지막 숨겨진 상태를 문장 임베딩으로 사용합니다.",
    "마지막으로 문장 임베딩을 사용하여 텍스트를 요약할 수 있습니다.",
    "이를 수행하는 한 가지 방법은 각 문장과 다른 문장 간의 유사도를 계산하고 가장 높은 유사도 점수를 가진 문장을 선택하는 것입니다. ",
    "이를 위해 코사인 유사성 측정을 사용합니다.",
             ]
embeddings = embed_text(model=bi_encoder1, paragraphs=paragraphs, return_tensor=False)
print(f'len(embeddings):{len(embeddings)}')


len(embeddings):19


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

# cosine 계산
similarity_matrix = cosine_similarity(embeddings)

# 요약 생성
# => 첫번째 문장과 유사한 문장 3개를 더해서 추출요약문 작성
num_sentences = 2  # 추가할 문장수
summary_sentences = []
summary_sentences.append(paragraphs[0]) # 첫번째 문장을 담음.

for i in range(num_sentences):
    sentence_scores = list(enumerate(similarity_matrix[i]))
    sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)
    summary_sentences.append(paragraphs[sentence_scores[1][0]])
    summary = ' '.join(summary_sentences)
    
print(summary)  

Text 요약은 중요한 정보를 유지하면서 텍스트 문서의 길이를 줄이는 프로세스입니다. 마지막으로 문장 임베딩을 사용하여 텍스트를 요약할 수 있습니다. 추출 요약을 위한 인기 있는 접근 방식 중 하나는 BERT로 , 이는 텍스트 요약 작업을 위해 미세 조정할 수 있는 사전 훈련된 언어 모델입니다.
